In [2]:
#  this script is meant to deal with the data of 8 recognition runs and generate models saved in corresponding folder
'''
input:
    cfg.session=ses1
    cfg.modelFolder=f"{cfg.subjects_dir}/{cfg.subjectName}/{cfg.session}_recognition/clf/"
    cfg.dataFolder=f"{cfg.subjects_dir}/{cfg.subjectName}/{cfg.session}_recognition/"
output:
    models in cfg.modelFolder
'''


import os
import sys
sys.path.append('/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/')
import argparse
import numpy as np
import nibabel as nib
import scipy.io as sio
from subprocess import call
from nibabel.nicom import dicomreaders
import pydicom as dicom  # type: ignore
import time
from glob import glob
import shutil
from nilearn.image import new_img_like
import joblib
import rtCommon.utils as utils
from rtCommon.utils import loadConfigFile
from rtCommon.fileClient import FileInterface
import rtCommon.projectUtils as projUtils
from rtCommon.imageHandling import readRetryDicomFromFileInterface, getDicomFileName, convertDicomImgToNifti


# argParser = argparse.ArgumentParser()
# argParser.add_argument('--config', '-c', default='sub001.ses1.toml', type=str, help='experiment file (.json or .toml)')
# args = argParser.parse_args()
from rtCommon.cfg_loading import mkdir,cfg_loading
cfg = cfg_loading("sub001.ses1.toml")

sys.path.append('/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/expScripts/recognition/')
from recognition_dataAnalysisFunctions import recognition_preprocess,minimalClass,behaviorDataLoading



In [6]:
'''
This script is adapted from classRegion.py
Purpose:
    to train and save the classifiers for all ROIs

'''
 

'''
from the recognition exp dir, run batchRegions.sh, it will run the script classRegion.sh, which is just a feeder for classRegion.py for all ROI/parcels across both wang and schaefer.

classRegion.py simply runs a runwise cross-validated classifier across the runs of recognition data, then stores the average accuracy of the ROI it was assigned in an numpy array. 
This is stored within the subject specific folder (e.g. wang2014/0111171/output/roi25_rh.npy )

input:
    1 subject: which subject
    2 dataloc: neurosketch or realtime
    3 roiloc: schaefer2018 or wang2014
    4 roinum: number of rois you want
    5 roihemi: which hemisphere

'''
import nibabel as nib
import numpy as np
import os
import sys
import time
import pandas as pd
from sklearn.linear_model import LogisticRegression

# What subject are you running
subject = "sub001" #sys.argv[1]
dataSource = "realtime"
recognition_dir = '/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/subjects/sub001/ses1/recognition/' #sys.argv[1]

print("NO ROI LOCATION ENTERED: Using radius of wang2014")
roiloc = "wang"

print("NO DATASOURCE ENTERED: Using original neurosketch data")
dataSource = 'neurosketch'

print("NO ROI SPECIFIED: Using roi number 1")
roinum="1"

if roiloc == "wang2014":
    try:
        roihemi = "_{}".format("lh")
        print("Since this is wang2014, we need a hemisphere, in this case {}".format(roihemi))
    except:
        print("this is wang 2014, so we need a hemisphere, but one was not specified")
        assert 1 == 2
else:
    roihemi=""

print("Running subject {}, with {} as a data source, {} roi #{} {}".format(subject, dataSource, roiloc, roinum, roihemi))


NO ROI LOCATION ENTERED: Using radius of wang2014
NO DATASOURCE ENTERED: Using original neurosketch data
NO ROI SPECIFIED: Using roi number 1
Since this is wang2014, we need a hemisphere, in this case _lh
Running subject sub001, with neurosketch as a data source, wang2014 roi #1 _lh


In [18]:

# dataSource depending, there are a number of keywords to fill in: 
# ses: which day of data collection
# run: which run number on that day (single digit)
# phase: 12, 34, or 56
# sub: subject number
if dataSource == "neurosketch":
    funcdata = "/gpfs/milgram/project/turk-browne/jukebox/ntb/projects/sketchloop02/subjects/{sub}_neurosketch/data/nifti/realtime_preprocessed/{sub}_neurosketch_recognition_run_{run}.nii.gz"
    metadata = "/gpfs/milgram/project/turk-browne/jukebox/ntb/projects/sketchloop02/data/features/recog/metadata_{sub}_V1_{phase}.csv"
    anat = "/gpfs/milgram/project/turk-browne/jukebox/ntb/projects/sketchloop02/subjects/{sub}_neurosketch/data/nifti/{sub}_neurosketch_anat_mprage_brain.nii.gz"
elif dataSource == "realtime":
    funcdata = "{recognition_dir}run{run}.nii.gz"
    metadata = "{recognition_dir}{subject}_{run_i}.csv"
    anat = "$TO_BE_FILLED"
else:
    funcdata = "/gpfs/milgram/project/turk-browne/projects/rtTest/searchout/feat/{sub}_pre.nii.gz"
    metadata = "/gpfs/milgram/project/turk-browne/jukebox/ntb/projects/sketchloop02/data/features/recog/metadata_{sub}_V1_{phase}.csv"
    anat = "$TO_BE_FILLED"
    
outloc = "/gpfs/milgram/project/turk-browne/projects/rtTest/searchout"
starttime = time.time()


def Wait(waitfor, delay=1):
    while not os.path.exists(waitfor):
        time.sleep(delay)
        print('waiting for {}'.format(waitfor))
        
def normalize(X):
    X = X - X.mean(3)
    return X

def Class(data, bcvar):
    metas = bcvar[0]
    data4d = data[0]
    print(data4d.shape)

    accs = []
    for run in range(6):
        testX = data4d[run]
        testY = metas[run]
        trainX = data4d[np.arange(6) != run]
        trainX = trainX.reshape(trainX.shape[0]*trainX.shape[1], -1)
        trainY = []
        for meta in range(6):
            if meta != run:
                trainY.extend(metas[run])
        clf = LogisticRegression(penalty='l2',C=1, solver='lbfgs', max_iter=1000, 
                                 multi_class='multinomial').fit(trainX, trainY)
                
        # Monitor progress by printing accuracy (only useful if you're running a test set)
        acc = clf.score(testX, testY)
        accs.append(acc)
    
    return np.mean(accs)


# phasedict = dict(zip([1,2,3,4,5,6,7,8],["12", "12", "34", "34", "56", "56"]))
phasedict = dict(zip([1,2,3,4,5,6,7,8],[cfg.actualRuns]))
imcodeDict={"A": "bed", "B": "Chair", "C": "table", "D": "bench"}

mask = nib.load(f"{cfg.mask_dir}{roiloc}_roi{roinum}{roihemi}.nii.gz").get_data()
mask = mask.astype(int)# say some things about the mask.
print('mask dimensions: {}'. format(mask.shape))
print('number of voxels in mask: {}'.format(np.sum(mask)))


mask dimensions: (64, 64, 36)
number of voxels in mask: 271


/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/ipykernel_launcher.py:61: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


In [46]:
run_i,run=0,cfg.actualRuns[0]
print(run, end='--')
# retrieve from the dictionary which phase it is, assign the session
# Build the path for the preprocessed functional data
this4d = f"{cfg.recognition_dir}run{run}.nii.gz" # run data

# Read in the metadata, and reduce it to only the TR values from this run, add to a list
thismeta = pd.read_csv(f"{cfg.recognition_dir}{cfg.subjectName}_{run_i+1}.csv")
# thismeta = thismeta[thismeta['run_num'] == int(run)]
TR_num = list(thismeta.TR.astype(int))
labels = list(thismeta.Item)
labels = [None if type(label)==float else imcodeDict[label] for label in labels]


2--

In [48]:
print("LENGTH OF TR: {}".format(len(TR_num)))
# Load the functional data
runIm = nib.load(this4d)
affine_mat = runIm.affine
runImDat = runIm.get_data()


LENGTH OF TR: 140


/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  """


In [49]:
# Use the TR numbers to select the correct features
features = [runImDat[:,:,:,n+2] for n in TR_num]
features = np.array(features)
features = features[:, mask==1]
print("shape of features", features.shape, "shape of mask", mask.shape)
featmean = features.mean(1)[..., None]
features = features - featmean
features = np.expand_dims(features, 0)

shape of features (140, 271) shape of mask (64, 64, 36)


In [74]:
# Compile preprocessed data and corresponding indices
metas = []
runs=[]
for run_i,run in enumerate(cfg.actualRuns):
    print(run, end='--')
    # Build the path for the preprocessed functional data
    this4d = f"{cfg.recognition_dir}run{run}.nii.gz" # run data
    
    # Read in the metadata, and reduce it to only the TR values from this run, add to a list
    thismeta = pd.read_csv(f"{cfg.recognition_dir}{cfg.subjectName}_{run_i+1}.csv")

    TR_num = list(thismeta.TR.astype(int))
    labels = list(thismeta.Item)
    labels = [None if type(label)==float else imcodeDict[label] for label in labels]


    print("LENGTH OF TR: {}".format(len(TR_num)))
    # Load the functional data
    runIm = nib.load(this4d)
    affine_mat = runIm.affine
    runImDat = runIm.get_data()
    
    # Use the TR numbers to select the correct features
    features = [runImDat[:,:,:,n+2] for n in TR_num]
    features = np.array(features)
    features = features[:, mask==1]
    print("shape of features", features.shape, "shape of mask", mask.shape)
    featmean = features.mean(1)[..., None]
    features = features - featmean
    
    # Append both so we can use it later
    metas.append(labels)
    runs.append(features) # if run_i == 0 else np.concatenate((runs, features))

2--LENGTH OF TR: 140


/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


shape of features (140, 271) shape of mask (64, 64, 36)
3--LENGTH OF TR: 148
shape of features (148, 271) shape of mask (64, 64, 36)
4--LENGTH OF TR: 147
shape of features (147, 271) shape of mask (64, 64, 36)
5--LENGTH OF TR: 143
shape of features (143, 271) shape of mask (64, 64, 36)
6--LENGTH OF TR: 146
shape of features (146, 271) shape of mask (64, 64, 36)
7--LENGTH OF TR: 148
shape of features (148, 271) shape of mask (64, 64, 36)
9--LENGTH OF TR: 139
shape of features (139, 271) shape of mask (64, 64, 36)
10--LENGTH OF TR: 145
shape of features (145, 271) shape of mask (64, 64, 36)


In [117]:
print(trainX.shape)
print(len(trainY))

(1016, 271)
980


In [138]:

def Class(data, bcvar):
    metas = bcvar
    data4d = data
    accs = []
    for curr_run in range(8):
        testX = data4d[curr_run]
        testY = metas[curr_run]
        trainX=None
        for train_run in range(8):
            if train_run!=curr_run:
                trainX = data4d[train_run] if type(trainX)!=np.ndarray else np.concatenate((trainX, data4d[train_run]),axis=0)
        trainY = []
        for train_run in range(8):
            if train_run!=curr_run:
                trainY.extend(metas[train_run])
        # remove nan type
        id=[type(i)==str for i in trainY]
        trainY=[i for i in trainY if type(i)==str]
        trainX=trainX[id]

        clf = LogisticRegression(penalty='l2',C=1, solver='lbfgs', max_iter=1000, 
                                 multi_class='multinomial').fit(trainX, trainY)

        # Monitor progress by printing accuracy (only useful if you're running a test set)
        id=[type(i)==str for i in testY]
        testY=[i for i in testY if type(i)==str]
        testX=testX[id]
        acc = clf.score(testX, testY)
        accs.append(acc)
    return np.mean(accs)
accs=Class(data, bcvar)

/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative 

In [137]:
accs

[0.5833333333333334,
 0.6875,
 0.4791666666666667,
 0.5833333333333334,
 0.5833333333333334,
 0.5833333333333334,
 0.5416666666666666,
 0.3958333333333333]

In [148]:

command=f"bash {cfg.recognition_expScripts_dir}batchRegions.sh sub001.ses1.toml"
command

'bash /gpfs/milgram/project/turk-browne/projects/rtSynth_rt/expScripts/recognition/batchRegions.sh sub001.ses1.toml'

In [146]:
cfg.recognition_expScripts_dir

'/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/expScripts/recognition/'

In [149]:
f"{cfg.mask_dir}{roiloc}_{roinum}{roihemi}.nii.gz"

'/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/subjects/sub001/ses1/recognition/mask/wang_1_lh.nii.gz'

In [153]:
brain=np.load(f"{cfg.recognition_dir}brain_run10.npy")
print(brain.shape)
mask=nib.load(f"{cfg.recognition_dir}chosenMask.nii.gz").get_data()
print(mask.shape)

(47, 64, 64, 36)
(64, 64, 36)


/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  This is separate from the ipykernel package so we can avoid doing imports until
